In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Cargar el dataset
#input_file = "reviews_proc.csv"
input_file = "reviews_prod_20_29.csv"
data = pd.read_csv(input_file, sep=';', encoding="utf-8")

# Suponiendo que las reseñas están en la columna 'review'
reseñas = data['review'].astype(str).fillna("")

# Vectorización BoW
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(reseñas)

# Convertir la matriz BoW en un DataFrame
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Agregar las estrellas al DataFrame
bow_df['stars'] = data['stars']

# Guardar en un nuevo CSV
bow_df.to_csv("reviews_bow.csv", index=False, sep=';')

#print(data)
#print(reseñas)
#print(X_bow)
print(bow_df)
print("Archivo reviews_bow.csv generado con éxito.")

     00  00stream  100  19  19loaded  1xplayback  31  45  5l  61  ...  ángulo  \
0     0         0    0   0         0           0   0   0   0   0  ...       0   
1     0         0    0   0         0           0   0   0   0   0  ...       0   
2     0         0    0   0         0           0   0   0   0   0  ...       0   
3     0         0    0   0         0           0   0   0   0   0  ...       0   
4     0         0    0   0         0           0   0   0   0   0  ...       0   
..   ..       ...  ...  ..       ...         ...  ..  ..  ..  ..  ...     ...   
113   0         0    0   0         0           0   0   0   0   0  ...       0   
114   0         0    0   0         0           0   0   0   0   0  ...       0   
115   0         0    0   0         0           0   0   0   0   0  ...       0   
116   0         0    0   0         0           0   0   0   0   0  ...       0   
117   0         0    0   0         0           0   0   0   0   0  ...       0   

     él  és  éxito  último 

# **ENTRENAMIENTO DEL MODELO**

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


X_train, X_test, Y_train, Y_test = train_test_split(X_bow, data['stars'], test_size=0.2, random_state=12)

def entrenar_modelo(XTrain,XTest,YTrain,YTest):
    clasificadores = [
        MultinomialNB(),
        SVC(random_state = 12),
        LogisticRegression(random_state=12),
        DecisionTreeClassifier(criterion='entropy', random_state=12),  # Árbol de decisión base
        BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy', random_state=12),
                         n_estimators=15, random_state=12),  # Bagging
        AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy', random_state=12),
                           n_estimators=15, random_state=12,algorithm = 'SAMME'),  # Boosting
        RandomForestClassifier(criterion='entropy', max_features='log2', n_estimators=15, random_state=12)  # Random Forest
    ]
    nombres = ['Naive-Bayes','SVC','LogisticRegression','DecisionTree', 'Bagging', 'Boosting', 'RandomForest']
    for i, clasificador in enumerate(clasificadores):
        #Entrenar modelo
        clasificador.fit(XTrain, YTrain)
        #Realizar prediccion
        y_test_pred = clasificador.predict(XTest)
        #Evaluar modelo
        acc = round(accuracy_score(YTest, y_test_pred)*100,2)
        print("El accuracy del clasificador ",nombres[i]," es: ",acc)
    
    return acc

acc = entrenar_modelo(X_train,X_test,Y_train,Y_test)

El accuracy del clasificador  Naive-Bayes  es:  62.5
El accuracy del clasificador  SVC  es:  70.83
El accuracy del clasificador  LogisticRegression  es:  75.0
El accuracy del clasificador  DecisionTree  es:  70.83
El accuracy del clasificador  Bagging  es:  75.0
El accuracy del clasificador  Boosting  es:  66.67
El accuracy del clasificador  RandomForest  es:  75.0
